Copyright 2020 Philipp Ollendorff

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
import itertools
import copy
import numpy as np
import random
import time
import math
import matplotlib
from PIL import Image
from matplotlib import pyplot as plt

# Verification, Solving and Reducing Puzzles

In [ ]:
# Verification
def trios_not_present(row):
    count = 0
    ele = row[0]
    for i in range(len(row)):
        if row[i] == 2:
            count = 0
            continue
        elif row[i] == ele:
            count += 1
        else:
            count = 1
            ele = row[i]
        if count == 3:
            return False
    return True

def equal_amounts_of_01(row):
    n_half = int(len(row) // 2)
    if (np.count_nonzero(row == 0) <= n_half) and (np.count_nonzero(row == 1) <= n_half):
        return True
    return False
        
def valid_puzzle(p, printing=False):
    if p is None:
        return False
    valid = True
    row_sets = []
    col_sets = []
    
    for row in p:
        valid = valid & trios_not_present(row)
    if printing:
        print('Trios rows: {}'.format(valid))
    for row in p:
        valid = valid & equal_amounts_of_01(row)
    if printing:
        print('Equals rows: {}'.format(valid))
    for row in p:
        if 2 not in row:
            row_sets.append(tuple(row))
    valid = valid & (len(row_sets) == len(set(row_sets)))
    if printing:
        print('Unique rows: {}'.format(valid))
    
    for col in np.transpose(p):
        valid = valid & trios_not_present(col)
    if printing:
        print('Trios cols: {}'.format(valid))
    for col in np.transpose(p):
        valid = valid & equal_amounts_of_01(col)
    if printing:
        print('Equals cols: {}'.format(valid))
    for col in np.transpose(p):
        if 2 not in col:
            col_sets.append(tuple(col))
    valid = valid & (len(col_sets) == len(set(col_sets)))
    
    if printing:
        print('Unique cols: {}'.format(valid))
    return valid
    
# Solving
def solve_backtracking(puzzle, index=0):
    n = len(puzzle)
    if index == n*n:
        return puzzle
    else:
        choices = [0, 1]
        random.shuffle(choices) # Shuffling necessary to generate unique puzzles
        for value in choices:
            puzzle[index // n][index % n] = value
            if valid_puzzle(puzzle):
                result = solve_backtracking(puzzle, index + 1)
                if valid_puzzle(result):
                    return result
            else:
                for i in range(index, n*n):
                    puzzle[i // n][i % n] = 2
        return
                
# Reducing
def reset_random_digits(puzzle, k):
    size = len(puzzle) ** 2
    if k > size:
        print('Error, would remove everything')
        return puzzle
    list_of_resets = []
    while len(list_of_resets) < k:
        index = int(random.random() * size)
        if index not in list_of_resets and puzzle[index // n][index % n] != 2:
            list_of_resets.append(index)
            puzzle[index // n][index % n] = 2
    return puzzle

# Human solving techniques

In [ ]:
'''
Human solving algorithms to see if randomly reduced Takuzu is still solvable without guessing.
'''
    
# If the same number is visited twice, it cannot occur a third time
# So each side gets filled with the opposite number
def duos(puzzle, i):
    n = len(puzzle)
    indices_0 = []
    indices_1 = []
    count = 0
    ele = -1
    for j in range(n):
        if puzzle[i][j] == 2:
            count = 0
            continue
        if puzzle[i][j] == ele:
            count += 1
        else:
            count = 1
            ele = puzzle[i][j]
        if count == 2: 
            if ele == 0:
                if 0 <= j - 2:
                    indices_0.append(j - 2)
                if j + 1 < len(puzzle):
                    indices_0.append(j + 1)
            if ele == 1:
                if 0 <= j - 2:
                    indices_1.append(j - 2)
                if j + 1 < len(puzzle):
                    indices_1.append(j + 1)
    for x in indices_0:
        puzzle[i][x] = 1
    for y in indices_1:
        puzzle[i][y] = 0
    
    return puzzle

        
# All rows and columns need to be unique
def unique(puzzle, i):
    n = len(puzzle)
    for j in range(n):
        if (puzzle[i] == puzzle[j]).all():
            continue

        equalities = puzzle[i] == puzzle[j]
        if not sum(puzzle[i] == puzzle[j]) == n - 2:
            continue
        if np.count_nonzero(puzzle[i] == 2) > 2 or np.count_nonzero(puzzle[j] == 2) > 2:
            continue

        cond1 = len(np.where(puzzle[i][np.where(equalities == False)] != 2)[0]) == 0
        cond2 = len(np.where(puzzle[j][np.where(equalities == False)] != 2)[0]) == 0
        if cond1 and cond2:
            empty_indices = np.where(puzzle[i] == 2)
            first = empty_indices[0][0]
            second = empty_indices[0][1]
            first_j = empty_indices[0][0]
            second_j = empty_indices[0][1]
            puzzle[i][first] = 1
            puzzle[i][second] = 0
            puzzle[j][first_j] = 0
            puzzle[j][second_j] = 1

            if not valid_puzzle(puzzle):
                # Switch back
                puzzle[i][first] = 0
                puzzle[i][second] = 1
                puzzle[j][first_j] = 1
                puzzle[j][second_j] = 0

        elif cond1 or cond2:
            if cond1:
                k = i
            else:
                k = j
            empty_indices = np.where(puzzle[k] == 2)
            first = empty_indices[0][0]
            second = empty_indices[0][1]
            puzzle[k][first] = 1
            puzzle[k][second] = 0

            if not valid_puzzle(puzzle):
                # Switch back
                puzzle[k][first] = 0
                puzzle[k][second] = 1
    return puzzle

# Avoid the same number three times in a row
def trios(puzzle, i):
    n = len(puzzle)
    for j in range(1, n - 1):
        if puzzle[i][j] == 2 and puzzle[i][j-1] == puzzle[i][j+1] and puzzle[i][j-1] != 2:
            puzzle[i][j] = 1 - puzzle[i][j-1] 

    if np.count_nonzero(puzzle[i] == 2) == 3:
        for k in range(n - 3):
            if puzzle[i][k] == puzzle[i][k+3] and puzzle[i][k + 1] == puzzle[i][k + 2] == 2:
                if np.count_nonzero(puzzle[i] == puzzle[i][k]) == int(n // 2) - 2:
                    index_last_two = (set(np.where(puzzle[i] == 2)[0]) ^ set([k+1, k+2])).pop()
                    puzzle[i][index_last_two] = puzzle[i][k]
    return puzzle

                    
# Complete an almost finished line
def complete_line(puzzle, i):
    n = len(puzzle)
    n_half = int(n // 2)
    if np.count_nonzero(puzzle[i] == 0) == n_half:
        for j in range(n):
            if puzzle[i][j] == 2:
                puzzle[i][j] = 1

    elif np.count_nonzero(puzzle[i] == 1) == n_half:
        for j in range(n):
            if puzzle[i][j] == 2:
                puzzle[i][j] = 0
    return puzzle

def solve_human(puzzle):
    def solve_anonymous(puzzle):
        n = len(puzzle)
        for i in range(n):
            puzzle = duos(puzzle, i)
            puzzle = unique(puzzle, i)
            puzzle = complete_line(puzzle, i)
            puzzle = trios(puzzle, i)
        return puzzle

    puzzle = apply_f_until_done(puzzle, f=solve_anonymous)
    return puzzle

# Iteratively applies function f on the puzzle until it cannot be solved further
def apply_f_until_done(puzzle, f):
    original = copy.copy(puzzle)
    puzzle = f(puzzle)
    puzzle = np.transpose(puzzle)
    puzzle = f(puzzle)
    puzzle = np.transpose(puzzle)
    a = time.time()
    while not (original == puzzle).all():
        puzzle = np.transpose(puzzle)
        original = copy.copy(puzzle)
        puzzle = f(puzzle)
    puzzle = np.transpose(puzzle)
    puzzle = f(puzzle)
    return puzzle


def is_deterministic(puzzle):
    #get a random empty spot
    row_index = int(random.random() * len(puzzle))
    empty_occurences = np.where(puzzle == 2)
    i = empty_occurences[0][0]
    j = empty_occurences[1][0]
    deterministic = False
    for k in [0,1]:
        tester = copy.copy(puzzle)
        tester[i][j] = k
        deterministic = deterministic ^ valid_puzzle(solve_backtracking(tester, 0))
    return deterministic

# Human removal techniques

In [ ]:
# Remove obvious redundancies like Duos and Trios
def remove(puzzle):
    n = len(puzzle)
    def removal(puzzle):
        for i in range(n):
            
            # Duos
            indices_0 = []
            count = 0
            ele = -1
            for j in range(n):
                if puzzle[i][j] == 2:
                    count = 0
                    continue
                if puzzle[i][j] == ele:
                    count += 1
                else:
                    count = 1
                    ele = puzzle[i][j]
                if count == 2: 
                    if 0 <= j - 2:
                        puzzle[i][j - 2] = 2
                    if j + 1 < len(puzzle):
                        puzzle[i][j + 1] = 2
                
                
        for i in range(n):
            # Trios
            for j in range(1, n - 1):
                if puzzle[i][j-1] == puzzle[i][j+1] and puzzle[i][j-1] != 2:
                    puzzle[i][j] = 2
        return puzzle
        
    puzzle = apply_f_until_done(puzzle, f=removal)
    return puzzle
    
def count_empties(puzzle):
    return np.count_nonzero(puzzle == 2)

def visual_grid(puzzle, save=False, name=""):
    p = copy.copy(puzzle).astype(dtype='object')
    n = len(puzzle)
    p[np.where(p == 2)] = None
    plt.figure() #figsize=(20,10))
    grid = plt.table(cellText=p, cellLoc='center', loc='center', bbox = [0,0,1,1.5])
    grid.set_fontsize(18)
    plt.axis('off')
    plt.tight_layout()
    if save:
        plt.savefig("{}x{}_{}.jpg".format(n, n, name), bbox_inches='tight')
        plt.close()
    else:
        plt.show()
        plt.close()

# Puzzle creation pipeline

In [ ]:
puzzle_index = 500
save_it = False

In [ ]:
%%time
nr_of_puzzles = 1

for i in range(nr_of_puzzles):
    puzzle_index += 1
    print('Preparing puzzle no. {}'.format(puzzle_index))
    n = 10
    threshold = -1
    while threshold < 65:
        print('.', end='')
        empty_puzzle = np.asarray([2] * n*n).reshape(n, n)
        solved_puzzle = solve_backtracking(empty_puzzle, 0)
        base_candidate = remove(solved_puzzle)

        #cand = copy.copy(base_candidate)
        empties = 0
        max_tries = 500
        counter = 0
        still_solvable_candidate = np.asarray([])
        while empties < 15 and counter < max_tries:
            empties = 1
            solved = copy.copy(base_candidate)
            candidate = copy.copy(base_candidate)
            while 2 not in solve_human(solved):
                still_solvable_candidate = copy.copy(candidate)
                candidate = reset_random_digits(candidate, 1)
                solved = copy.copy(candidate)
                empties += 1
            counter += 1
        if still_solvable_candidate.any():
            candidate = still_solvable_candidate
        threshold = count_empties(candidate)
    print('\nWhich resulted in a puzzle with {} empty slots.'.format(threshold))
    visual_grid(cand, save=save_it, name='Hard_{}_{}'.format(count_empties(candidate), puzzle_index))

# Print all puzzles into collated pdf

In [ ]:
import os
puzzles = []
for entry in os.scandir("/home/pollendo/02_Coding/Takuzu/"):
    if ".jpg" in entry.path:
        puzzles.append(entry.path)
puzzles = sorted(puzzles)
for a in range(len(puzzles)): # 216 = 36 pages * 6 puzzles per page.
    print(puzzles[a])

In [ ]:
imglist = []
page = 1
for entry in puzzles:
    print(entry)
    # get images    
    img = Image.open(entry)
    imglist.append(copy.copy(img))
    if len(imglist) == 6:
        
        # get width and height, adjust for better format
        w, h = imglist[0].size
        w += 20
        h += 20
        
        # create big empty image with place for images
        new_image = Image.new('L', (w*2 + 50, h*3 + 50), color=255)

        # put images on new_image
        for i in range(2):
            for j in range(3):
                new_image.paste(imglist[i * 3 + j], (i*w + 35, h*j + 35))

        # save it
        new_image.save("/home/pollendo/02_Coding/Takuzu/Page_{}_.pdf".format(page))
        page += 1
        imglist = []